In [14]:
!pip install dash --user

In [15]:
!pip install dash_auth --user

In [16]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import dash_auth

In [17]:
url = "https://github.com/mavimelato/visualizacaodados/blob/main/dados.xlsx?raw=true"
df = pd.read_excel(url, engine='openpyxl')
df.head()

,Unnamed: 0,ID_Pedido,Data_Pedido,ID_Representante,Nome_Representante,Regional,ID_Produto,Nome_Produto,Valor_Produto,Quantidade_Vendida,Valor_Total_Venda,Nome_Cliente,Cidade_Cliente,Estado_Cliente
0,NaN,10001,2020-01-01,1004,Isabela Carolina,Sudeste,SKU9005,Celular Modelo 3,1800,21,37800,amazon,Rio de Janeiro,RJ
1,NaN,10002,2020-01-01,1004,Isabela Carolina,Sudeste,SKU9005,Celular Modelo 3,1800,57,102600,Shoptime,Betim,MG
2,NaN,10003,2020-01-02,1002,Lucas Souza,Sudeste,SKU9011,Notebook Modelo 3,4300,59,253700,Carrefour,Rio de Janeiro,RJ
3,NaN,10004,2020-01-02,1002,Lucas Souza,Sudeste,SKU9011,Notebook Modelo 3,4300,89,382700,Carrefour,Rio de Janeiro,RJ
4,NaN,10005,2020-01-02,1005,Tais Fernandes,Sudeste,SKU9008,Mouse,88,71,6248,Ricardo Eletro,Vitória,ES


In [18]:
# PRIMEIRO GRÁFICO DA NÚMERO 1

# Criando o app Dash e configurando autenticação para os usuários
app = Dash(__name__)

df['Data_Pedido'] = pd.to_datetime(df['Data_Pedido'])

df['Mes'] = df['Data_Pedido'].dt.strftime('%B')

# Especificar a ordem dos meses
ordem_meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Agrupar os dados por mês e somar as quantidades vendidas
df_vendas_por_mes = df.groupby('Mes')['Quantidade_Vendida'].sum().reset_index()
df_vendas_por_mes['Mes'] = pd.Categorical(df_vendas_por_mes['Mes'], categories=ordem_meses, ordered=True)
df_vendas_por_mes = df_vendas_por_mes.sort_values('Mes')


fig = px.line(df_vendas_por_mes, x='Mes', y='Quantidade_Vendida',
              title='Quantidade Total Vendida por Mês',
              labels={'Quantidade_Vendida': 'Quantidade Vendida', 'Mes': 'Mês'},
              template='plotly_dark')


app.layout = html.Div([
    html.Div(children='''
        Dashboard de Vendas
    '''),
    
    html.H3(children="Quantidade Total Vendida por Mês", id="subtitulo"),
    
    dcc.Graph(id='vendas_por_mes', figure=fig)
    
], style={"text-align": "center"})

if __name__ == '__main__':
    app.run_server(debug=True)

In [19]:
# LETRA B DA NÚMERO 1

# Criando o app Dash e configurando autenticação para os usuários
app = Dash(__name__)

df['Data_Pedido'] = pd.to_datetime(df['Data_Pedido'])

df['Mes'] = df['Data_Pedido'].dt.strftime('%B')

# Especificar a ordem dos meses
ordem_meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Agrupar os dados por mês e somar as quantidades vendidas
df_vendas_por_mes = df.groupby('Mes')['Quantidade_Vendida'].sum().reset_index()
df_vendas_por_mes['Mes'] = pd.Categorical(df_vendas_por_mes['Mes'], categories=ordem_meses, ordered=True)
df_vendas_por_mes = df_vendas_por_mes.sort_values('Mes')

# Agrupar os dados por representante e somar as vendas de cada um
df_vendas_por_representante = df.groupby('Nome_Representante')['Quantidade_Vendida'].sum().reset_index()

# Criar o gráfico de barras
fig_representante = px.bar(df_vendas_por_representante, x='Nome_Representante', y='Quantidade_Vendida',
                            title='Total de Vendas por Representante',
                            labels={'Quantidade_Vendida': 'Quantidade Vendida', 'Nome_Representante': 'Representante'},
                            template='plotly_dark')


app.layout = html.Div([
    html.Div(children='''
        Dashboard de Vendas
    '''),
    
    html.H3(children="Quantidade Total Vendida por Mês", id="subtitulo"),
    
    dcc.Graph(id='vendas_por_mes', figure=fig),
    
    html.H3(children="Total de Vendas por Representante"),
    
    dcc.Graph(id='vendas_por_representante', figure=fig_representante)
    
], style={"text-align": "center"})

if __name__ == '__main__':
    app.run_server(debug=True)


In [20]:
# LETRA C DA NÚMERO 1

from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import dash_auth
from dash import dash_table

# Criando o app Dash e configurando autenticação para os usuários
app = Dash(__name__)

df['Data_Pedido'] = pd.to_datetime(df['Data_Pedido'])
df['Mes'] = df['Data_Pedido'].dt.strftime('%B')

# Agrupar os dados por mês e somar as quantidades vendidas
df_vendas_por_mes = df.groupby('Mes')['Quantidade_Vendida'].sum().reset_index()

# Agrupar os dados por representante e somar as vendas de cada um
df_vendas_por_representante = df.groupby('Nome_Representante')['Quantidade_Vendida'].sum().reset_index()

# Agrupar os dados por produto e somar as vendas de cada um
df_vendas_por_produto = df.groupby(['ID_Produto', 'Nome_Produto'])['Quantidade_Vendida'].sum().reset_index()

# Criar o gráfico de barras
fig_representante = px.bar(df_vendas_por_representante, x='Nome_Representante', y='Quantidade_Vendida',
                            title='Total de Vendas por Representante',
                            labels={'Quantidade_Vendida': 'Quantidade Vendida', 'Nome_Representante': 'Representante'},
                            template='plotly_dark')

# Criar a tabela de vendas por produto
tabela_vendas_por_produto = dash_table.DataTable(
    id='tabela_vendas_por_produto',
    columns=[{"name": i, "id": i} for i in df_vendas_por_produto.columns],
    data=df_vendas_por_produto.to_dict('records'),
    style_table={'overflowX': 'auto'}
)

app.layout = html.Div([
    html.Div(children='''
        Dashboard de Vendas
    '''),
    
    html.H3(children="Quantidade Total Vendida por Mês", id="subtitulo"),
    
    dcc.Graph(id='vendas_por_mes', figure=fig),
    
    html.H3(children="Total de Vendas por Representante"),
    
    dcc.Graph(id='vendas_por_representante', figure=fig_representante),
    
    html.H3(children="Total de Vendas por Produto"),
    
    tabela_vendas_por_produto
    
], style={"text-align": "center"})

if __name__ == '__main__':
    app.run_server(debug=True)


In [21]:
# LETRA D DA 1

from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import dash_auth

# Criando o app Dash e configurando autenticação para os usuários
app = Dash(__name__)

# Agrupar os dados por estado do cliente e somar as quantidades vendidas
df_vendas_por_estado = df.groupby('Estado_Cliente')['Quantidade_Vendida'].sum().reset_index()

# Criar o gráfico de pizza
fig_estado = px.pie(df_vendas_por_estado, values='Quantidade_Vendida', names='Estado_Cliente', title='Total de Vendas por Regional')

app.layout = html.Div([
    html.Div(children='''
        Dashboard de Vendas
    '''),
    
    html.H3(children="Quantidade Total Vendida por Mês", id="subtitulo"),
    
    dcc.Graph(id='vendas_por_mes', figure=fig),
    
    html.H3(children="Total de Vendas por Representante"),
    
    dcc.Graph(id='vendas_por_representante', figure=fig_representante),
    
    html.H3(children="Total de Vendas por Produto"),
    
    tabela_vendas_por_produto,
    
    html.H3(children="Total de Vendas por Regional"),
    
    dcc.Graph(id='vendas_por_estado', figure=fig_estado)
    
], style={"text-align": "center"})

if __name__ == '__main__':
    app.run_server(debug=True)

In [22]:
## último gráfico da 1

from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import dash_auth

# Criando o app Dash 
app = Dash(__name__)

# Criar o gráfico de barras com paleta de cores personalizada e maior contraste
fig_barra = px.bar(df, x='Estado_Cliente', y='Quantidade_Vendida',
                   barmode='group',
                   title='Total de Vendas por Estado',
                   labels={'Quantidade_Vendida': 'Quantidade Vendida', 'Estado_Cliente': 'Estado'},
                   
                   )

# Layout do app Dash
app.layout = html.Div([
    html.Div(children='Dashboard de Vendas'),
    html.H3(children="Total de vendas por estado", id="subtitulo"),
    dcc.Graph(id="total_vendas_estado", figure=fig_barra)
], style={"text-align": "center"})

if __name__ == '__main__':
    app.run_server(debug=True)


In [23]:
##NÚMERO 2:

from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import dash_auth

# Criando o app Dash 
app = Dash(__name__)

df['Mes'] = df['Data_Pedido'].dt.strftime('%B')

# Especificar a ordem dos meses
ordem_meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
ordem_meses.append("Todos")

lista_regional = list(df["Regional"].unique())
lista_regional.append("Todos")

lista_produtos = list(df["Nome_Produto"].unique())
lista_produtos.append("Todos")

lista_representante = list(df["Nome_Representante"].unique())
lista_representante.append("Todos")

# Agrupar os dados por mês e somar as quantidades vendidas
df_vendas_por_mes = df.groupby('Mes')['Quantidade_Vendida'].sum().reset_index()
df_vendas_por_mes['Mes'] = pd.Categorical(df_vendas_por_mes['Mes'], categories=ordem_meses, ordered=True)
df_vendas_por_mes = df_vendas_por_mes.sort_values('Mes')

df_vendas_por_regiao = df.groupby('Regional')['Quantidade_Vendida'].sum().reset_index()
df_vendas_por_regiao['Regional'] = pd.Categorical(df_vendas_por_regiao['Regional'], categories=lista_regional, ordered=True)
df_vendas_por_regiao = df_vendas_por_regiao.sort_values('Regional')

df_vendas_por_rep = df.groupby('Nome_Representante')['Quantidade_Vendida'].sum().reset_index()


# Primeiro, agrupe o DataFrame original por 'Nome_Produto' e 'Mes' e some a 'Quantidade_Vendida'
df_agrupado = df.groupby(['Nome_Produto', 'Mes'])['Quantidade_Vendida'].sum().reset_index()
df_agrupado['Mes'] = pd.Categorical(df_agrupado['Mes'], categories=ordem_meses, ordered=True)
df_agrupado = df_agrupado.sort_values('Mes')

# Agrupar o DataFrame original por 'Nome_Produto' e 'Regiao', somando a quantidade vendida
df_agrupado2 = df.groupby(['Nome_Produto', 'Regional'])['Quantidade_Vendida'].sum().reset_index()

df_agrupado3 = df.groupby(['Nome_Produto','Nome_Representante'])['Quantidade_Vendida'].sum().reset_index()

fig1 = px.bar(df_vendas_por_mes, x='Mes', y='Quantidade_Vendida',
              title='Quantidade Total Vendida por Mês',
              labels={'Quantidade_Vendida': 'Quantidade Vendida', 'Mes': 'Mês'},
              template='plotly_dark', color="Mes", barmode="group")

fig2= px.bar(df_vendas_por_regiao, x="Regional", y="Quantidade_Vendida", color="Regional", barmode="group")

fig3 = px.bar(df_vendas_por_rep, x="Nome_Representante", y="Quantidade_Vendida", color="Nome_Representante", barmode="group")



app.layout = html.Div(children=[
    html.H1(children='Visualização de Dados'),
    
    html.Div(children='''
        Dashboard de Vendas'''),
    
    html.H3(children="Vendas de cada Produto", id="subtitulo"),
    
    html.Div(children=[
        dcc.Dropdown(lista_produtos, value="Todos", id='selecao_produto'),
    ], style={"width": "50%", "margin": "auto"}),
    
    dcc.Graph(id='vendas_produtos',figure=fig1),
    
    html.Div(children=[
        dcc.Dropdown(lista_regional, value="Todos", id='selecao_regional'),
    ], style={"width": "50%", "margin": "auto"}),
    
    dcc.Graph(id='vendas_regiao', figure=fig2),

    html.Div(children=[
        dcc.Dropdown(lista_representante, value="Todos", id='selecao_rep'),
    ], style={"width": "50%", "margin": "auto"}),
    
    dcc.Graph(id='vendas_rep', figure=fig3),
    
    
], style={"text-align": "center"})

@app.callback(
    Output('subtitulo','children'),
    Output('vendas_produtos','figure'),
    Output('vendas_regiao','figure'),
    Output('vendas_rep','figure'),
    Input('selecao_produto','value'),
    Input('selecao_regional','value'),
    Input('selecao_rep','value')
)

def selecionar(produto, regiao, representante):
    if produto == "Todos" and regiao == "Todos" and representante == "Todos":  #verufica as 2 condições ao mesmo tempo 
        texto = "Vendas de cada Produto"
        fig1 = px.bar(df_vendas_por_mes, x='Mes', y='Quantidade_Vendida',
              title='Quantidade Total Vendida por Mês',
              labels={'Quantidade_Vendida': 'Quantidade Vendida', 'Mes': 'Mês'},
              template='plotly_dark', color="Mes", barmode="group")
        
        fig2= px.bar(df_vendas_por_regiao, x="Regional", y="Quantidade_Vendida", color="Regional", barmode="group")
        
        fig3 = px.bar(df_vendas_por_rep, x="Nome_Representante", y="Quantidade_Vendida", color="Nome_Representante", barmode="group", template='plotly_dark')
    else:
        df_filtrada = df_agrupado
        df_filtrada2 = df_agrupado2
        df_filtrada3 = df_agrupado3
        if produto != "Todos":
            #filtrar de acordo com a marca
            df_filtrada = df_filtrada.loc[df_filtrada['Nome_Produto']==produto, :]
        if regiao != "Todos":
            # filtrar de acordo com o pais
            df_filtrada2 = df_filtrada2.loc[df_filtrada2["Regional"]==regiao, :]
        if representante != "Todos":
            df_filtrada3 = df_filtrada3.loc[df_filtrada3["Nome_Representante"]==representante, :]
        
        texto = f"Vendas do Produto de nome {produto}"
        fig1 = px.bar(df_filtrada, x='Mes', y='Quantidade_Vendida',
              title='Quantidade Total Vendida por Mês',
              labels={'Quantidade_Vendida': 'Quantidade Vendida', 'Mes': 'Mês'},
              template='plotly_dark', color="Mes", barmode="group")
        
        fig2= px.bar(df_filtrada2, x="Nome_Produto", y="Quantidade_Vendida", color="Regional", barmode="group")
        
        fig3 = px.bar(df_filtrada3, x="Nome_Produto", y="Quantidade_Vendida", color="Nome_Produto", barmode="group", template="plotly_dark" )
    return texto, fig1, fig2, fig3

if __name__ == '__main__':
    app.run_server(debug=False)

In [27]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import dash_auth
import dash

app = dash.Dash(__name__)

url = "https://github.com/mavimelato/visualizacaodados/blob/main/dados.xlsx?raw=true"
df = pd.read_excel(url, engine='openpyxl')

# Opções de estados

estados_options = [{'label': estado, 'value': estado} for estado in df['Estado_Cliente'].unique()]
app = dash.Dash(__name__)

# Layout do aplicativo
app.layout = html.Div([
    html.Label('Estado do Cliente'),
    dcc.Dropdown(id='estado-dropdown', options=estados_options, value=None),
    html.Label('Cidade do Cliente'),
    dcc.Dropdown(id='cidade-dropdown', value=None),
    dcc.Graph(id='grafico-vendas')
])

# Atualiza as opções da lista de cidades conforme o estado selecionado

@app.callback(
    Output('cidade-dropdown', 'options'),
    [Input('estado-dropdown', 'value')]
)

def update_cities_options(selected_estado):
    if selected_estado is None:
        return []
    else:
        cidades_options = [{'label': cidade, 'value': cidade} for cidade in df[df['Estado_Cliente'] == selected_estado]['Cidade_Cliente'].unique()]
        return cidades_options

# Atualiza o gráfico conforme o estado e a cidade selecionados

@app.callback(
    Output('grafico-vendas', 'figure'),
    [Input('estado-dropdown', 'value'),
     Input('cidade-dropdown', 'value')]
)

def update_graph(selected_estado, selected_cidade):
    if selected_estado is None or selected_cidade is None:
        return {}
    filtered_df = df[(df['Estado_Cliente'] == selected_estado) & (df['Cidade_Cliente'] == selected_cidade)]
    fig = px.bar(filtered_df, x='Nome_Produto', y='Quantidade_Vendida', title=f'Vendas em {selected_cidade}, {selected_estado}')
    return fig

# Executa o aplicativo
if __name__ == '__main__':
    app.run_server(debug=True)

In [28]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import dash_auth

# Criando o app Dash e configurando autenticação para os usuários
app = Dash(__name__)

url = "https://github.com/mavimelato/visualizacaodados/blob/main/dados.xlsx?raw=true"
df = pd.read_excel(url, engine='openpyxl')

df['Data_Pedido'] = pd.to_datetime(df['Data_Pedido'])
df['Mes'] = df['Data_Pedido'].dt.strftime('%B')

# Especificar a ordem dos meses
ordem_meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
ordem_meses.append("Todos")

# Lista de opções para filtros
lista_regional = list(df["Regional"].unique())
lista_regional.append("Todos")

lista_produtos = list(df["Nome_Produto"].unique())
lista_produtos.append("Todos")

lista_representante = list(df["Nome_Representante"].unique())
lista_representante.append("Todos")

# Agrupar os dados por mês e somar as quantidades vendidas
df_vendas_por_mes = df.groupby('Mes')['Quantidade_Vendida'].sum().reset_index()
df_vendas_por_mes['Mes'] = pd.Categorical(df_vendas_por_mes['Mes'], categories=ordem_meses, ordered=True)
df_vendas_por_mes = df_vendas_por_mes.sort_values('Mes')

df_vendas_por_regiao = df.groupby('Regional')['Quantidade_Vendida'].sum().reset_index()
df_vendas_por_regiao['Regional'] = pd.Categorical(df_vendas_por_regiao['Regional'], categories=lista_regional, ordered=True)
df_vendas_por_regiao = df_vendas_por_regiao.sort_values('Regional')

df_vendas_por_rep = df.groupby('Nome_Representante')['Quantidade_Vendida'].sum().reset_index()

# Layout do aplicativo
app.layout = html.Div([
    html.H1(children='Dashboard de Vendas', style={'textAlign': 'center'}),

    html.Div(children='''
        Análise de Dados de Vendas
    ''', style={'textAlign': 'center'}),

    html.Div([
        html.H3(children="Vendas de cada Produto", id="subtitulo"),
        dcc.Graph(id='vendas_produtos'),
        dcc.Dropdown(options=[{'label': produto, 'value': produto} for produto in lista_produtos],
                     value="Todos", id='selecao_produto', style={'width': '50%', 'margin': 'auto'}),
    ]),

    html.Div([
        html.H3(children="Total de Vendas por Região"),
        dcc.Graph(id='vendas_regiao'),
        dcc.Dropdown(options=[{'label': regiao, 'value': regiao} for regiao in lista_regional],
                     value="Todos", id='selecao_regional', style={'width': '50%', 'margin': 'auto'}),
    ]),

    html.Div([
        html.H3(children="Total de Vendas por Representante"),
        dcc.Graph(id='vendas_rep'),
        dcc.Dropdown(options=[{'label': rep, 'value': rep} for rep in lista_representante],
                     value="Todos", id='selecao_rep', style={'width': '50%', 'margin': 'auto'}),
    ]),

    html.Div([
        html.H3(children="Vendas por Estado e Cidade"),
        html.Label('Estado do Cliente'),
        dcc.Dropdown(id='estado-dropdown', options=[{'label': estado, 'value': estado} for estado in df['Estado_Cliente'].unique()], value=None),
        html.Label('Cidade do Cliente'),
        dcc.Dropdown(id='cidade-dropdown', value=None),
        dcc.Graph(id='grafico-vendas'),
    ]),

    html.Div([
        html.H3(children="Total de Vendas por Mês"),
        dcc.Graph(id='vendas_por_mes', figure=px.bar(df_vendas_por_mes, x='Mes', y='Quantidade_Vendida',
                                                      title='Quantidade Total Vendida por Mês', labels={'Quantidade_Vendida': 'Quantidade Vendida', 'Mes': 'Mês'},
                                                      template='plotly_dark', color="Mes", barmode="group")),
    ]),

    html.Div([
        html.H3(children="Total de Vendas por Região"),
        dcc.Graph(id='vendas_por_regiao', figure=px.bar(df_vendas_por_regiao, x="Regional", y="Quantidade_Vendida",
                                                         color="Regional", barmode="group")),
    ]),

    html.Div([
        html.H3(children="Total de Vendas por Representante"),
        dcc.Graph(id='vendas_por_representante', figure=px.bar(df_vendas_por_rep, x="Nome_Representante", y="Quantidade_Vendida",
                                                                color="Nome_Representante", barmode="group")),
    ]),

    html.Div([
        html.H3(children="Total de Vendas por Estado"),
        dcc.Graph(id='vendas_por_estado', figure=px.pie(df.groupby('Estado_Cliente')['Quantidade_Vendida'].sum().reset_index(),
                                                        values='Quantidade_Vendida', names='Estado_Cliente',
                                                        title='Total de Vendas por Estado')),
    ])
])

# Callback para atualizar as opções de cidades com base no estado selecionado
@app.callback(
    Output('cidade-dropdown', 'options'),
    [Input('estado-dropdown', 'value')]
)
def update_cities_options(selected_estado):
    if selected_estado is None:
        return []
    else:
        cidades_options = [{'label': cidade, 'value': cidade} for cidade in df[df['Estado_Cliente'] == selected_estado]['Cidade_Cliente'].unique()]
        return cidades_options

# Callback para atualizar o gráfico conforme o estado e a cidade selecionados
@app.callback(
    Output('grafico-vendas', 'figure'),
    [Input('estado-dropdown', 'value'),
     Input('cidade-dropdown', 'value')]
)
def update_graph(selected_estado, selected_cidade):
    if selected_estado is None or selected_cidade is None:
        return {}
    filtered_df = df[(df['Estado_Cliente'] == selected_estado) & (df['Cidade_Cliente'] == selected_cidade)]
    fig = px.bar(filtered_df, x='Nome_Produto', y='Quantidade_Vendida', title=f'Vendas em {selected_cidade}, {selected_estado}')
    return fig

# Callback para atualizar os gráficos de vendas conforme as seleções de filtros
@app.callback(
    [Output('subtitulo', 'children'),
     Output('vendas_produtos', 'figure'),
     Output('vendas_regiao', 'figure'),
     Output('vendas_rep', 'figure')],
    [Input('selecao_produto', 'value'),
     Input('selecao_regional', 'value'),
     Input('selecao_rep', 'value')]
)
def selecionar(produto, regiao, representante):
    if produto == "Todos" and regiao == "Todos" and representante == "Todos":
        texto = "Vendas de cada Produto"
        fig1 = px.bar(df_vendas_por_mes, x='Mes', y='Quantidade_Vendida',
                      title='Quantidade Total Vendida por Mês', labels={'Quantidade_Vendida': 'Quantidade Vendida', 'Mes': 'Mês'},
                      template='plotly_dark', color="Mes", barmode="group")

        fig2 = px.bar(df_vendas_por_regiao, x="Regional", y="Quantidade_Vendida", color="Regional", barmode="group")

        fig3 = px.bar(df_vendas_por_rep, x="Nome_Representante", y="Quantidade_Vendida", color="Nome_Representante", barmode="group")
    else:
        df_filtrada = df.groupby(['Nome_Produto', 'Mes'])['Quantidade_Vendida'].sum().reset_index()
        df_filtrada2 = df.groupby(['Nome_Produto', 'Regional'])['Quantidade_Vendida'].sum().reset_index()
        df_filtrada3 = df.groupby(['Nome_Produto', 'Nome_Representante'])['Quantidade_Vendida'].sum().reset_index()
        if produto != "Todos":
            df_filtrada = df_filtrada.loc[df_filtrada['Nome_Produto'] == produto, :]
        if regiao != "Todos":
            df_filtrada2 = df_filtrada2.loc[df_filtrada2["Regional"] == regiao, :]
        if representante != "Todos":
            df_filtrada3 = df_filtrada3.loc[df_filtrada3["Nome_Representante"] == representante, :]

        texto = f"Vendas do Produto de nome {produto}"
        fig1 = px.bar(df_filtrada, x='Mes', y='Quantidade_Vendida',
                      title='Quantidade Total Vendida por Mês', labels={'Quantidade_Vendida': 'Quantidade Vendida', 'Mes': 'Mês'},
                      template='plotly_dark', color="Mes", barmode="group")

        fig2 = px.bar(df_filtrada2, x="Nome_Produto", y="Quantidade_Vendida", color="Regional", barmode="group")

        fig3 = px.bar(df_filtrada3, x="Nome_Produto", y="Quantidade_Vendida", color="Nome_Produto", barmode="group", template="plotly_dark")
    return texto, fig1, fig2, fig3

# Executa o aplicativo
if __name__ == '__main__':
    app.run_server(debug=True)